<a href="https://colab.research.google.com/github/ucfilho/raianars_fly/blob/master/RTC_teste_03_fev_12_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from random import randint

In [0]:
'''
Computers and Chemical Engineering 117 (2018) 82–96
Stream Tin(K) Tout(K) CP(kW/K)
H1      533   433     3.0
H2      523   403     1.5
C1      453   508     2.0
C2      393   513     4.0
HU      553   552     -
CU      303   353     -

U = 0.2 kW/( m 2 · K ) for all matches. Exchanger cost
($) = 300 · A**0.5 for all matches. CHU = 110 $/(kW ·year).
CCU = 12.2 $/(kW ·year).
'''

'\nComputers and Chemical Engineering 117 (2018) 82–96\nStream Tin(K) Tout(K) CP(kW/K)\nH1      533   433     3.0\nH2      523   403     1.5\nC1      453   508     2.0\nC2      393   513     4.0\nHU      553   552     -\nCU      303   353     -\n\nU = 0.2 kW/( m 2 · K ) for all matches. Exchanger cost\n($) = 300 · A**0.5 for all matches. CHU = 110 $/(kW ·year).\nCCU = 12.2 $/(kW ·year).\n'

In [0]:
'''
H1 com C2: 533      --> 454.89
H1 com C1: 454.89   --> 433

H2 com C1: 523      --> 448.53
H2 com C2: 448.53   --> 421.76

'''

'\nH1 com C2: 533      --> 454.89\nH1 com C1: 454.89   --> 433\n\nH2 com C1: 523      --> 448.53\nH2 com C2: 448.53   --> 421.76\n\n'

In [0]:
'''
NE: numero de estagios
Hot: numero de correntes quentes
Cold: numero de correntes frias
CPhot e CPcold: capacidades de trocas termicas (massa*calor especifico)
Thot: temperaturas iniciais das correntes quentes
Thot_target: temperatura final esperada (hot)
Tcold_target: temperatura final esperada (cold)
Alfa: matriz que contem os fatores para chegar nas temperaturas hot em cada estagio
     (nao foi incluido aqui as associadas as utilidades q sao Thot_target)
x: valores e Alfa sem ser colocado no formato matricial e tambem o TRC
     (a lista x tera sempre 2*NE elementos sendo q alguns podem ser nao usados)
TRC: representa qual correte fria troca com a quente 
    (serve para calcular as temperaturas das cold)
'''
# dados de entrada dos codigo
NE=2
Hot=2
Cold=2
Num=NE*Hot
Thot=np.array([533,523])
Tcold=np.array([393,403])
CPhot=np.array([3.0,1.5])
CPcold=np.array([2,4])
Thot_target=np.array([433,453])
Tcold_target=np.array([508,513])
x=np.array([0.853452157598499,0.9518784761151048,0.8576099426386232,0.9403161438476803,2,1,1,2])


In [0]:
Alfa=x[0:Num].reshape((2, 2))
TRC=x[(Num):(2*Num)].reshape((2, 2))

print('matriz de temperaturas')
print(Alfa)
print('matriz quais correntes quentes trocam com quais frias')
print(TRC)

matriz de temperaturas
[[0.85345216 0.95187848]
 [0.85760994 0.94031614]]
matriz quais correntes quentes trocam com quais frias
[[2. 1.]
 [1. 2.]]


In [0]:
'''
HotStream: rotina q chega nos valores das temperaturas da corrente quente ao 
           longo dos estagios....
Alfa: matriz com fatores q Thot sera corrigida
NE: numero de estagios-incluir dois para representar todas temperaturas
    pq nao foi considerado as utililidades e temperatura inicial
Thot: temperatura quente inicial
Thot_target: temperatura quente final
Hot: numero de correntes quentes
'''
def HotStream(Alfa,NE,Thot,Thot_target,Hot):
  Hot_Final=np.zeros((NE+2,Hot))
  Hot_Final[0,:]=np.copy(Thot)
  for i in range(Hot):
    for j in range(NE):
      k=i+1
      Hot_Final[k,j]=Hot_Final[i,j]*Alfa[j,i]
  Hot_Final[NE+1,:]=np.copy(Thot_target)
  return Hot_Final
      

In [0]:
Thot_calc=HotStream(Alfa,NE,Thot,Thot_target,Hot)
print(Thot_calc)
'''
a ultima linha de Thot_calc representa a temperatura apos a utilidade, mas caso 
  nao utilize utilidades a ultima e penultima linhas se tornam iguais
'''

[[533.   523.  ]
 [454.89 448.53]
 [433.   421.76]
 [433.   453.  ]]


'\na ultima linha de Thot_calc representa a temperatura apos a utilidade, mas caso \n  nao utilize utilidades a ultima e penultima linhas se tornam iguais\n'

In [0]:
'''
ColdStream: rotina q chega nos valores de temperatura da corrente cold
            deve ser chamada depois de HotStream pq precisa dos valores 
            de HotStream (Thot_calc)
Thot_calc: perfil de temperatura utilizado no calculo de Tcold_calc
CPhot e CPcold: capacidades de trocas termicas (massa*calor especifico)
Tcold_calc: perfil de temperatura de saida do ColdStream
TRC: matriz que orienta as escolhas das correntes torcas de calor
Flag, Teste : teste logico para evitar q uma corrente cold troque no mesmo 
          estagio calor com duas correntes quentes 
Tcold_target: temperatura cold final
Cold: numero de correntes frias
Tcold: temperatura inicial das correntes frias
'''
def ColdStream(NE,Hot,Cold,TRC,Tcold,Thot_calc,Tcold_target,CPhot,CPcold):
  Teste=np.zeros(Cold)
  Cold_Final=np.zeros((NE+2,Cold))
  Cold_Final[0,:]=np.copy(Tcold)
  for j in range(NE):
    for i in range(Cold):
      ih=int(TRC[i,j]-1) # ih indice da hot q troca calor (subtrai 1 pq comeca zero)
      print("%d %d %d"%(i,j,ih))
      Flag=Teste[ih]
      k=i+1
      Tf=Thot_calc[k,j] # temperatura final da hot na troca
      Ti=Thot_calc[i,j] # temperatura inicial da hot na troca
      if(Flag==0): # so trocar uma corrente por estagio
        Teste[ih]=Teste[ih]+1
        Cold_Final[k,j]=Cold_Final[i,j]-CPhot[ih]*(Tf-Ti)/CPcold[i]
      else: # caso q nao troca calor pq a corrente ja trocou
        Cold_Final[k,j]=Cold_Final[i,j]
  Cold_Final[NE+1,:]=np.copy(Tcold_target)
  return Cold_Final

In [0]:
Tcold_calc=ColdStream(NE,Hot,Cold,TRC,Tcold,Thot_calc,Tcold_target,CPhot,CPcold)
print(Tcold_calc)

0 0 1
1 0 0
0 1 0
1 1 1
[[393.     403.    ]
 [451.5825 403.    ]
 [468.     403.    ]
 [508.     513.    ]]
